## Use case #1: using file inputParams.json (recommended use)

Execute the following cells only the first time running this notebook:

In [1]:
import os
os.chdir("..")

Loading the input parameters file in a new instance of the `controller`:

In [2]:
from synthDataGen.controller import Controller

controller = Controller()
controller.loadMainParams("inputParams.json", "./synthDataGen/settings/")
df = controller.getDataFromSource()
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-17 20:00:00,41.05,78.51,43.54,51.17,64.49,69.25,70.58,71.02,81.73,54.21,60.27,70.35,58.92,42.13,105.59,251.37
2023-07-17 21:00:00,38.46,67.43,42.44,47.23,62.08,61.77,62.47,65.32,78.12,50.60,56.69,66.85,56.53,40.24,99.69,240.87
2023-07-17 22:00:00,34.16,61.92,39.69,47.38,62.34,60.20,62.31,65.02,76.21,48.84,56.14,63.96,58.04,36.50,95.68,207.98
2023-07-17 23:00:00,31.99,57.85,37.90,44.71,60.05,56.79,53.58,61.57,73.94,41.62,50.13,60.23,53.42,32.38,92.80,183.87
2023-07-18 00:00:00,29.69,53.33,35.51,40.17,54.39,52.53,43.55,54.16,69.04,39.43,47.37,58.25,48.97,29.80,85.46,187.92
2023-07-18 01:00:00,29.01,53.42,34.33,41.50,48.78,50.06,37.09,51.79,67.58,39.26,46.71,58.70,47.93,29.39,90.98,191.61
2023-07-18 02:00:00,27.73,52.82,32.82,37.52,49.00,49.01,36.54,50.75,63.42,38.71,46.35,58.05,47.96,28.79,91.24,191.94
2023-07-18 03:00:00,28.31,54.15,31.34,37.83,53.98,49.73,42.93,51.53,60.48,39.30,45.94,58.61,49.25,28.83,91.10,193.77
2023-07-18 04:00:00,31.57,61.93,31.49,34.14,57.79,54.41,51.14,60.11,59.71,40.98,47.72,64.40,49.20,28.71,90.21,213.18


Getting the first DataFrame. Then adjust it with the anual adjustments dictionary and resample it to the desired resolution:

In [3]:
from synthDataGen.controller import AdjustmentController

adjustmentController = AdjustmentController(controller.inputJSON)
df = adjustmentController.performAnualAdjustments(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-17 20:00:00,45.155,86.361,47.894,56.287,70.939,76.175,77.638,78.122,89.903,59.631,66.297,77.385,64.812,46.343,116.149,276.507
2023-07-17 21:00:00,42.306,74.173,46.684,51.953,68.288,67.947,68.717,71.852,85.932,55.660,62.359,73.535,62.183,44.264,109.659,264.957
2023-07-17 22:00:00,37.576,68.112,43.659,52.118,68.574,66.220,68.541,71.522,83.831,53.724,61.754,70.356,63.844,40.150,105.248,228.778
2023-07-17 23:00:00,35.189,63.635,41.690,49.181,66.055,62.469,58.938,67.727,81.334,45.782,55.143,66.253,58.762,35.618,102.080,202.257
2023-07-18 00:00:00,32.659,58.663,39.061,44.187,59.829,57.783,47.905,59.576,75.944,43.373,52.107,64.075,53.867,32.780,94.006,206.712
2023-07-18 01:00:00,31.911,58.762,37.763,45.650,53.658,55.066,40.799,56.969,74.338,43.186,51.381,64.570,52.723,32.329,100.078,210.771
2023-07-18 02:00:00,30.503,58.102,36.102,41.272,53.900,53.911,40.194,55.825,69.762,42.581,50.985,63.855,52.756,31.669,100.364,211.134
2023-07-18 03:00:00,31.141,59.565,34.474,41.613,59.378,54.703,47.223,56.683,66.528,43.230,50.534,64.471,54.175,31.713,100.210,213.147
2023-07-18 04:00:00,34.727,68.123,34.639,37.554,63.569,59.851,56.254,66.121,65.681,45.078,52.492,70.840,54.120,31.581,99.231,234.498


In [4]:
df = adjustmentController.downsample(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-17 20:00:00,43.7305,80.2670,47.2890,54.1200,69.6135,72.0610,73.1775,74.9870,87.9175,57.6455,64.3280,75.4600,63.4975,45.3035,112.904,270.7320
2023-07-17 22:00:00,36.3825,65.8735,42.6745,50.6495,67.3145,64.3445,63.7395,69.6245,82.5825,49.7530,58.4485,68.3045,61.3030,37.8840,103.664,215.5175
2023-07-18 00:00:00,32.2850,58.7125,38.4120,44.9185,56.7435,56.4245,44.3520,58.2725,75.1410,43.2795,51.7440,64.3225,53.2950,32.5545,97.042,208.7415
2023-07-18 02:00:00,30.8220,58.8335,35.2880,41.4425,56.6390,54.3070,43.7085,56.2540,68.1450,42.9055,50.7595,64.1630,53.4655,31.6910,100.287,212.1405
2023-07-18 04:00:00,37.5210,68.9040,35.3100,37.5375,63.6130,62.5790,60.2030,67.1715,65.1530,49.4505,55.1155,72.5505,55.7480,32.1805,99.165,242.1815


In [5]:
import pandas as pd
pd.set_option('display.precision', 2)

df = adjustmentController.upsample(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2023-07-17 20:00:00,43.73,80.27,47.29,54.12,69.61,72.06,73.18,74.99,87.92,57.65,64.33,75.46,63.50,45.30,112.90,270.73
2023-07-17 20:15:00,42.62,78.07,46.70,53.86,69.96,71.16,72.81,74.78,87.38,56.62,63.70,74.39,63.68,44.29,111.70,260.80
2023-07-17 20:30:00,41.56,75.98,46.11,53.54,70.12,70.24,72.22,74.44,86.81,55.61,63.04,73.37,63.74,43.30,110.50,251.73
2023-07-17 20:45:00,40.56,74.01,45.53,53.18,70.11,69.30,71.39,73.97,86.20,54.61,62.35,72.40,63.66,42.33,109.33,243.53
2023-07-17 21:00:00,39.62,72.16,44.95,52.77,69.91,68.34,70.32,73.37,85.56,53.62,61.63,71.48,63.45,41.39,108.16,236.19
2023-07-17 21:15:00,38.73,70.42,44.37,52.32,69.53,67.37,69.03,72.63,84.87,52.64,60.88,70.61,63.11,40.48,107.02,229.72
2023-07-17 21:30:00,37.89,68.79,43.80,51.81,68.97,66.38,67.50,71.76,84.15,51.67,60.10,69.79,62.64,39.59,105.88,224.12
2023-07-17 21:45:00,37.11,67.27,43.24,51.25,68.24,65.37,65.73,70.76,83.38,50.70,59.29,69.02,62.04,38.72,104.77,219.39
2023-07-17 22:00:00,36.38,65.87,42.67,50.65,67.31,64.34,63.74,69.62,82.58,49.75,58.45,68.30,61.30,37.88,103.66,215.52
2023-07-17 22:15:00,35.71,64.59,42.12,50.00,66.21,63.30,61.51,68.36,81.74,48.81,57.58,67.63,60.44,37.07,102.58,212.52


# Use case #2: passing parameters


In [6]:
from datetime import datetime

d = datetime(2023, 6, 5, 7, 0)

controller = Controller()
controller.loadMainParams("inputParams.json", "./synthDataGen/settings/")
df = controller.getDataFromSource(initialYear=2018, initDatetime=d, hoursAhead=6)
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,67.83,53.13,37.07,81.00,193.53
2023-06-05 07:00:00,69.11,50.74,37.06,81.43,177.57
2023-06-05 08:00:00,69.87,50.44,35.33,85.35,174.79
2023-06-05 09:00:00,69.09,50.50,34.96,85.31,177.76
2023-06-05 10:00:00,68.72,51.26,35.24,85.29,177.64
2023-06-05 11:00:00,69.03,49.11,35.13,85.36,176.71
2023-06-05 12:00:00,69.02,47.22,32.09,83.26,173.96


In [7]:
df = adjustmentController.performAnualAdjustments(df, adjustmentsDict={2018: 1.2, 2019: 2.3, 2020: 1.45, 2021: 3, 2022: 8})
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98
2023-06-05 10:00:00,69.54,52.44,35.75,87.85,191.85
2023-06-05 11:00:00,69.86,50.24,35.64,87.92,190.85
2023-06-05 12:00:00,69.85,48.31,32.56,85.76,187.88


In [8]:
df = adjustmentController.upsample(df, frequency="20T", method="spline", order=3)

import pandas as pd
pd.set_option('display.precision', 2)

df

,2018,2019,2020,2021,2022
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 06:20:00,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:40:00,69.70,52.60,36.97,84.39,196.71
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 07:20:00,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:40:00,70.36,51.62,36.27,86.21,189.39
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 08:20:00,70.36,51.59,36.12,87.20,188.97
2023-06-05 08:40:00,70.28,51.67,36.09,87.61,189.48
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98


In [9]:
df = adjustmentController.downsample(df, frequency="22.73T", aggregationFunc="mean")
df

,2018,2019,2020,2021,2022
2023-06-05 05:40:57.000,68.64,54.35,37.61,83.43,209.01
2023-06-05 06:03:40.800,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:26:24.600,69.70,52.60,36.97,84.39,196.71
2023-06-05 06:49:08.400,69.94,51.91,37.60,83.87,191.78
2023-06-05 07:11:52.200,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:34:36.000,70.36,51.62,36.27,86.21,189.39
2023-06-05 07:57:19.800,70.53,51.60,35.98,87.56,188.87
2023-06-05 08:20:03.600,70.28,51.67,36.09,87.61,189.48
2023-06-05 08:42:47.400,69.92,51.66,35.47,87.87,191.98
2023-06-05 09:05:31.200,70.02,51.84,36.01,88.18,190.97
